In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime

import pandas as pd
import numpy as np
from ta.volatility import BollingerBands
from ta.trend import MACD
from ta.momentum import RSIIndicator

In [2]:
# df.to_csv('btcusd.csv', index=False)

In [3]:
def apply_indicators(df):
    
    # Calculate the moving averages
    df['MA_daily'] = df['close'].rolling(window=50).mean()
    df['MA_weekly'] = df['close'].rolling(window=200).mean()

    # Calculate the Relative Strength Index (RSI)
    rsi_indicator = RSIIndicator(df['close'])
    df['RSI'] = rsi_indicator.rsi()

    # Calculate Bollinger Bands
    bollinger = BollingerBands(df['close'])
    df['BB_High'] = bollinger.bollinger_hband()
    df['BB_Low'] = bollinger.bollinger_lband()

    # Calculate MACD
    macd_indicator = MACD(df['close'])
    df['MACD'] = macd_indicator.macd()

    # Remove nas
    data = df.dropna()
    
    return data

# Getting data from the metatrader server

In [4]:
def get_data(symbol):

    # Getting data on the 15 minutes timeframe
    if not mt5.initialize():
        print("initialize() failed, error code =",mt5.last_error())
        quit()

    # set the symbol and timefram
    timeframe = mt5.TIMEFRAME_H1   # for one-minute bars

    #dates for retrieving the history
    date_from = datetime(2016, 12, 15)
    today = datetime.today()

    # get the history
    history = mt5.copy_rates_range(symbol, timeframe, date_from, today)

    if history is not None and len(history) > 0:
        # create DataFrame out of the obtained data
        rates_frame = pd.DataFrame(history).drop(['spread','real_volume'], axis =1)

        # convert time in seconds into the datetime format
        rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
    else:
        print("No data for the requested period")

    # terminate the connection to the MetaTrader 5 terminal
    mt5.shutdown()
    
    #adding indicators to retrieved data
    data = apply_indicators(rates_frame)
    data.to_csv(f'{symbol}.csv', index=False)
    
    return data

In [8]:
# some =get_data('BTCUSD')
data =get_data('XAUUSD')
data

,time,open,high,low,close,tick_volume,MA_daily,MA_weekly,RSI,BB_High,BB_Low,MACD
199,2017-01-31 11:00:00,1198.45,1204.32,1197.53,1204.23,26378,1192.6794,1192.27010,65.694013,1203.664980,1192.537020,2.179186
200,2017-01-31 12:00:00,1204.23,1206.82,1203.68,1204.89,38616,1193.1338,1192.57645,66.653721,1204.512757,1192.861243,2.460562
201,2017-01-31 13:00:00,1204.88,1205.65,1203.27,1204.11,37947,1193.5702,1192.88415,64.362141,1205.260365,1192.700635,2.590749
202,2017-01-31 14:00:00,1204.08,1214.34,1203.94,1212.94,70737,1194.1904,1193.23945,74.887745,1208.366068,1191.385932,3.367612
203,2017-01-31 15:00:00,1212.94,1215.41,1211.77,1212.51,75073,1194.7566,1193.59475,73.745489,1210.654529,1190.619471,3.903585
...,...,...,...,...,...,...,...,...,...,...,...,...
37757,2023-06-15 15:00:00,1953.14,1957.81,1951.95,1956.56,7423,1945.1786,1954.58930,63.967108,1954.626510,1923.696490,-0.000903
37758,2023-06-15 16:00:00,1956.56,1960.20,1956.56,1959.55,5943,1945.1422,1954.65475,66.026741,1957.597705,1922.186295,1.345662
37759,2023-06-15 17:00:00,1959.55,1960.28,1957.71,1958.76,4674,1945.2342,1954.71090,64.970058,1960.234958,1921.203042,2.322307
37760,2023-06-15 18:00:00,1958.78,1959.50,1956.93,1957.52,3697,1945.3326,1954.77385,63.258764,1962.211225,1920.577775,2.962102
